In [313]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [314]:
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 150)
%matplotlib inline

In [315]:
df = pd.read_pickle('/Users/andrewpeters/GitHub/fpl/data/interim/df.pkl')

To start, the main way I'll be adding features here is by making lag and rolling features. I also want to correct for that fact that not all data in each row is knowable _before_ the game. I know the fixture and the opponent before a game, but I don't know the minutes a player played until after, for example

In [316]:
prior_knowledge = ['player','team', 'position', 'gw', 'opponent_team', 'was_home', 'kickoff_time', 'season']
posterior_knowledge = ['team_h_score', 'team_a_score', 'minutes', 'goals_scored', 'assists', 'clean_sheets', 
                       'goals_conceded', 'bonus', 'bps', 'influence', 'creativity', 'threat', 'ict_index',
                      'transfers_balance', 'selected', 'transfers_in', 'transfers_out']

In [317]:
#ordering the columns so I can keep better track of this
#minutes are very much posterior knowledge as well, but I want to use this later to drop players who we could have easily predicted weren't going to play

df = df[['total_points'] + prior_knowledge + posterior_knowledge]

In [318]:
df = df.sort_values(['player', 'kickoff_time'])
shifted_df = df.groupby('player')[['total_points'] + posterior_knowledge].shift(1)
df = df.join(shifted_df, rsuffix='_prev').sort_values(['player', 'kickoff_time'])

In [319]:
posterior_knowledge_shift = [f'{col}_prev' for col in posterior_knowledge]

In [320]:
#generate rolling features, and join back with the original df. Using a minimum period of 4 to balance precision and not ending up with too many NaNs.

for window_size in tqdm(np.arange(2,11)):
    rolling_df = df.groupby('player')[['total_points_prev'] + posterior_knowledge_shift].rolling(window=window_size, min_periods=1).mean().reset_index().set_index('level_1')
    df = df.join(rolling_df[['total_points_prev'] + posterior_knowledge_shift], rsuffix = f'_{window_size}')

100%|██████████| 9/9 [00:13<00:00,  1.48s/it]


In [321]:
# at this point, I should drop the posterior_knowledge features -- the features we don't actually know until after the end of the gw
# i want to at least leave the 'minutes' col intact for the time being, because I might use this later for filter, or making the target value 'pts/minute'
posterior_knowledge.pop(2) 
df = df.drop(columns=posterior_knowledge)

In [322]:
# engineer features to show team-level performance
team_level = df.groupby(['team', 'season','gw']).sum(min_count=1) #min_count preserves NAs

#not all of the player-level features are helpful at the team-level; pick what I want to keep
team_level_keep_features = ['total_points', 'goals_scored', 'goals_conceded', 'clean_sheets', 'influence', 'creativity', 'threat']

#I want to keep all rolling window versions of the columns listed above:
team_level_cols = []
for col in list(team_level):
    for keep_col in team_level_keep_features:
        if keep_col in col:
            team_level_cols.append(col)
            
team_level = team_level[team_level_cols]
team_level = team_level.reset_index()

In [323]:
#merge the team-level features back in to the original df
df = df.merge(team_level, on = ['team', 'season', 'gw'], suffixes = ('', '_team'))

#do the same for opponents, so this can provide features on the difficulty of the opponent
df = df.merge(team_level, left_on = ['opponent_team', 'season', 'gw'], right_on = ['team', 'season', 'gw'], suffixes = ('', '_opponent'))

In [324]:
df.to_pickle('/Users/andrewpeters/GitHub/fpl/data/interim/df_with_rolling_features.pkl')

In [325]:
#save the team_level table so I can use later for forward-looking games
team_level.to_pickle('/Users/andrewpeters/GitHub/fpl/data/interim/team_level_rolling.pkl')

In [327]:
df[df['player'] == 'Mesut_Özil'].sort_values('kickoff_time').tail(5)

,total_points,player,team,position,gw,opponent_team,was_home,kickoff_time,season,minutes,total_points_prev,team_h_score_prev,team_a_score_prev,minutes_prev,goals_scored_prev,assists_prev,clean_sheets_prev,goals_conceded_prev,bonus_prev,bps_prev,influence_prev,creativity_prev,threat_prev,ict_index_prev,transfers_balance_prev,selected_prev,transfers_in_prev,transfers_out_prev,total_points_prev_2,team_h_score_prev_2,team_a_score_prev_2,minutes_prev_2,goals_scored_prev_2,assists_prev_2,clean_sheets_prev_2,goals_conceded_prev_2,bonus_prev_2,bps_prev_2,influence_prev_2,creativity_prev_2,threat_prev_2,ict_index_prev_2,transfers_balance_prev_2,selected_prev_2,transfers_in_prev_2,transfers_out_prev_2,total_points_prev_3,team_h_score_prev_3,team_a_score_prev_3,minutes_prev_3,goals_scored_prev_3,assists_prev_3,clean_sheets_prev_3,goals_conceded_prev_3,bonus_prev_3,bps_prev_3,influence_prev_3,creativity_prev_3,threat_prev_3,ict_index_prev_3,transfers_balance_prev_3,selected_prev_3,transfers_in_prev_3,transfers_out_prev_3,total_points_prev_4,team_h_score_prev_4,team_a_score_prev_4,minutes_prev_4,goals_scored_prev_4,assists_prev_4,clean_sheets_prev_4,goals_conceded_prev_4,bonus_prev_4,bps_prev_4,influence_prev_4,creativity_prev_4,threat_prev_4,ict_index_prev_4,transfers_balance_prev_4,selected_prev_4,transfers_in_prev_4,transfers_out_prev_4,total_points_prev_5,team_h_score_prev_5,team_a_score_prev_5,minutes_prev_5,goals_scored_prev_5,assists_prev_5,clean_sheets_prev_5,goals_conceded_prev_5,bonus_prev_5,bps_prev_5,influence_prev_5,creativity_prev_5,threat_prev_5,ict_index_prev_5,transfers_balance_prev_5,selected_prev_5,transfers_in_prev_5,transfers_out_prev_5,total_points_prev_6,team_h_score_prev_6,team_a_score_prev_6,minutes_prev_6,goals_scored_prev_6,assists_prev_6,clean_sheets_prev_6,goals_conceded_prev_6,bonus_prev_6,bps_prev_6,influence_prev_6,creativity_prev_6,threat_prev_6,ict_index_prev_6,transfers_balance_prev_6,selected_prev_6,transfers_in_prev_6,transfers_out_prev_6,total_points_prev_7,team_h_score_prev_7,team_a_score_prev_7,minutes_prev_7,goals_scored_prev_7,assists_prev_7,clean_sheets_prev_7,goals_conceded_prev_7,bonus_prev_7,bps_prev_7,influence_prev_7,creativity_prev_7,threat_prev_7,ict_index_prev_7,transfers_balance_prev_7,selected_prev_7,transfers_in_prev_7,transfers_out_prev_7,total_points_prev_8,team_h_score_prev_8,team_a_score_prev_8,minutes_prev_8,goals_scored_prev_8,assists_prev_8,clean_sheets_prev_8,goals_conceded_prev_8,bonus_prev_8,bps_prev_8,influence_prev_8,creativity_prev_8,threat_prev_8,ict_index_prev_8,transfers_balance_prev_8,selected_prev_8,transfers_in_prev_8,transfers_out_prev_8,total_points_prev_9,team_h_score_prev_9,team_a_score_prev_9,minutes_prev_9,goals_scored_prev_9,assists_prev_9,clean_sheets_prev_9,goals_conceded_prev_9,bonus_prev_9,bps_prev_9,influence_prev_9,creativity_prev_9,threat_prev_9,ict_index_prev_9,transfers_balance_prev_9,selected_prev_9,transfers_in_prev_9,transfers_out_prev_9,total_points_prev_10,team_h_score_prev_10,team_a_score_prev_10,minutes_prev_10,goals_scored_prev_10,assists_prev_10,clean_sheets_prev_10,goals_conceded_prev_10,bonus_prev_10,bps_prev_10,influence_prev_10,creativity_prev_10,threat_prev_10,ict_index_prev_10,transfers_balance_prev_10,selected_prev_10,transfers_in_prev_10,transfers_out_prev_10,total_points_team,total_points_prev_team,goals_scored_prev_team,clean_sheets_prev_team,goals_conceded_prev_team,total_points_prev_2_team,goals_scored_prev_2_team,clean_sheets_prev_2_team,goals_conceded_prev_2_team,influence_prev_2_team,creativity_prev_2_team,threat_prev_2_team,total_points_prev_3_team,goals_scored_prev_3_team,clean_sheets_prev_3_team,goals_conceded_prev_3_team,influence_prev_3_team,creativity_prev_3_team,threat_prev_3_team,total_points_prev_4_team,goals_scored_prev_4_team,clean_sheets_prev_4_team,goals_conceded_prev_4_team,influence_prev_4_team,creativity_prev_4_team,threat_prev_4_team,total_points_prev_5_team,goals_scored_prev_5_team,clean_sheets_prev_5_team,goal